# Mapa de calor das viagens de pedestres no município de São Paulo

---

In [200]:
import pandas as pd
import geopandas
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point

Carregando os datasets

In [201]:
od23_path = "../../data/OD/ODS/od23_all.csv" 
od = pd.read_csv(od23_path)

od_districts_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Distritos_2023_region.shp" 
od_districts = geopandas.read_file(od_districts_shapefile_path)

od_zones_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Zonas_2023.shp" 
od_zones = geopandas.read_file(od_zones_shapefile_path)

Filtrando zonas OD que pertencem ao município de SP

In [202]:
od_zones = od_zones.query('NumeroMuni == 36')

Convertendo CRS

In [203]:
od_districts = od_districts.to_crs("EPSG:4326")
od_zones = od_zones.to_crs("EPSG:4326")

In [204]:
od_districts.head()

,NumeroDist,NomeDistri,Area_ha,geometry
0,1,Água Rasa,715.05,"POLYGON Z ((-46.55407 -23.5685 0, -46.55413 -2..."
1,2,Alto de Pinheiros,751.26,"POLYGON Z ((-46.69236 -23.54638 0, -46.69283 -..."
2,3,Anhanguera,3339.95,"POLYGON Z ((-46.76109 -23.4274 0, -46.76114 -2..."
3,4,Aricanduva,686.26,"POLYGON Z ((-46.5018 -23.57948 0, -46.5018 -23..."
4,5,Artur Alvim,653.04,"POLYGON Z ((-46.47327 -23.54036 0, -46.47328 -..."


Filtrando viagens que originaram-se no município de São Paulo e pedestres

In [205]:
pedestrians_sp = od.query("MUNI_O == 36 and MODOPRIN == 17") # apenas SP e pedestres

Filtrando registros com coordenadas não válidas

In [206]:
pedestrians_coords = (
    pedestrians_sp[["CO_O_X", "CO_O_Y", "FE_VIA"]]
    .dropna()
    .query("CO_O_X != 0 and CO_O_Y != 0")  # remover coords nulas
)

Cálculo da representatividade do ponto em relação ao total de viagens a pé 

In [207]:
total_viagens = pedestrians_coords["FE_VIA"].sum()
pedestrians_coords["weight"] = pedestrians_coords["FE_VIA"] / total_viagens

## Mapa de calor

In [208]:
# Criando mapa base centralizado em São Paulo
sp_center = [-23.55, -46.63]
m = folium.Map(location=sp_center, zoom_start=11, tiles="CartoDB positron")

Cria mapa de calor com origens dos pedestres de SP

In [209]:
# Adicionando camada de calor ponderada por FE_VIA
heat_data = pedestrians_coords[["CO_O_X", "CO_O_Y", "weight"]].copy()

heat_map = HeatMap(heat_data.values.tolist(), radius=8, blur=15, max_zoom=1).add_to(m)

heat_data.head()

,CO_O_X,CO_O_Y,weight
7,-23.565856,-46.651093,0.000009
13,-23.549231,-46.637466,0.000014
14,-23.544407,-46.643665,0.000014
18,-23.569669,-46.645772,0.000014
19,-23.552317,-46.631656,0.000003


Adiciona polígonos das zonas OD como camada interativa

In [210]:
folium.GeoJson(
    od_zones,
    name="Zonas OD",
    style_function=lambda feature: {
        "fillColor": "transparent",
        "color": "green",
        "weight": 1,
        "fillOpacity": 0.1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NumeroZona", "NomeZona"],
        aliases=["Zona:", "Nome:"],
        sticky=False
    )
).add_to(m)

Intersects entre pontos e distritos para remover distritos que não pertencem ao município de São Paulo

In [211]:
# GeoDataFrame de pontos a partir das coordenadas de pedestres
gdf_points = geopandas.GeoDataFrame(
    pedestrians_coords,
    geometry=geopandas.points_from_xy(pedestrians_coords["CO_O_Y"], pedestrians_coords["CO_O_X"]),
    crs="EPSG:4326"
)

# spatial join: pontos x distritos
points_in_districts = geopandas.sjoin(gdf_points, od_districts, how="inner", predicate="intersects")

# apenas os distritos que tiveram interseção
od_districts_sp = od_districts.loc[
    od_districts.index.isin(points_in_districts["index_right"].unique())
].copy()

print(f"Número de distritos antes: {len(od_districts)}")
print(f"Número de distritos em SP (com viagens a pé): {len(od_districts_sp)}")


Número de distritos antes: 134
Número de distritos em SP (com viagens a pé): 100


Remover municípios que restaram: 'Diadema', 'Guarulhos', 'Osasco', 'São Caetano do Sul'

In [212]:
od_districts_sp = od_districts_sp.query(
    "NomeDistri not in ['Diadema', 'Guarulhos', 'Osasco', 'São Caetano do Sul']"
)

Adiciona polígonos dos distritos como camada interativa

In [213]:
folium.GeoJson(
    od_districts_sp,
    name="Distritos",
    style_function=lambda feature: {
        "fillColor": "transparent",
        "color": "red",
        "weight": 1,
        "fillOpacity": 0.05,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NumeroDist", "NomeDistri"],
        aliases=["Distrito:", "Nome:"],
        sticky=False
    ),
    show=False
).add_to(m)

Salva o mapa

In [214]:
# Controle de camadas
folium.LayerControl().add_to(m)

m.save("../built_maps/ped_heat_map.html")